In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

In [2]:
# train_path = "E:/casper/raw_data_training/test/"
# val_path = "E:/casper/raw_data_training/val/"
path = "E:/GLO_林口長庚資料集/Process_2022/Data/"

In [3]:
import os
# print(os.listdir(train_path + "MN"))

In [4]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir=path)

Resolving data files:   0%|          | 0/11399 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1376 [00:00<?, ?it/s]

Found cached dataset imagefolder (C:/Users/VLSI/.cache/huggingface/datasets/imagefolder/default-8787100763c9607f/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})

In [6]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

In [28]:
# from torchvision import datasets, models, transforms

# mean = np.array([0.5, 0.5, 0.5])
# std = np.array([0.25, 0.25, 0.25])

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std)
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std)
#     ]),
# }

In [29]:
# data_dir = "E:/casper/raw_data_training/test"
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=0)
#               for x in ['train', 'val']}

In [7]:
print(dataset)
prepared_ds = dataset.with_transform(transform)
print(prepared_ds)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})


In [8]:
from transformers import ViTForImageClassification

labels = ['IgA','MN']

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
model.to("cuda")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-gggg",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [10]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\VLSI\AppData\Local\Temp\ipykernel_25048\2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 11399
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1376
    })
})

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model.to("cuda"),
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=processor,
)

In [14]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\VLSI\anaconda3\envs\Tf-GPU\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.486700,0.947689,0.595203
200,0.594100,0.606814,0.676599
300,0.628300,0.673920,0.691134
400,0.474800,0.570172,0.706395
500,0.486400,0.569812,0.752907
600,0.503200,0.582865,0.742006
700,0.523800,0.516450,0.757994
800,0.429900,0.640440,0.695494
900,0.422000,0.562764,0.723110
1000,0.460500,0.577060,0.734738


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 3290665111GF
  train_loss               =       0.3731
  train_runtime            =   0:18:08.14
  train_samples_per_second =       41.903
  train_steps_per_second   =        2.621


In [15]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.7703
  eval_loss               =      0.507
  eval_runtime            = 0:00:13.00
  eval_samples_per_second =     105.78
  eval_steps_per_second   =     13.223


In [16]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
def Notification(subject,message):
    content = MIMEMultipart()  
    token = "dpyj vfsd fhhg fhfo"
    email = "casperliu376@gmail.com"
    content["subject"] = subject
    content["from"] = "VLSI machine"
    content["to"] = "casperliu1118@gmail.com"
    content.attach(MIMEText(message))
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:
        try:
            smtp.ehlo()
            smtp.starttls()
            smtp.login(email, token)
            smtp.send_message(content)
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)

In [17]:
mes = """HI CASPER,

    I AM TRANSFORMER, PLEASE CONTACT ME.

SINCERELY,
VLSI
"""
sub = "VIT"
Notification(sub,mes)

Complete!


In [18]:
print('hello')

hello


In [47]:
print(train_results.metrics)

{'train_runtime': 32451.7074, 'train_samples_per_second': 1.405, 'train_steps_per_second': 0.088, 'total_flos': 3.533324758466937e+18, 'train_loss': 0.36472809202394346, 'epoch': 4.0}
